In [29]:
import re


input_base="data"

labelselect=[1,7,8,12,18,21,47,84,88,94,97,107,112,196,208,231,283,301,309,310,327,508,650,656,665,668,671,892,928,937]
# 读取文件内容
with open('node.txt', 'r', encoding='utf-8') as file:
    content = file.read()

# 解析数据
data = []
pattern = r'\((\d+),\s*(\d+)\)'
matches = re.findall(pattern, content)

for match in matches:
    x = int(match[0])
    y = int(match[1])
    data.append((x, y))

print(data[0])

(51, 1)


In [15]:
def organize_files_by_label(input_file_path):
    # 创建一个字典来存储按标签分类的文件
    organized_data = {}

    try:
        # 读取输入文件
        with open(input_file_path, 'r') as file:
            for line in file:
                # 移除行末的换行符并按空格分割
                parts = line.strip().split()

                if len(parts) >= 2:
                    filename = parts[0]
                    label = parts[1]

                    # 如果标签已存在于字典中，添加文件名到对应的列表
                    if label in organized_data:
                        organized_data[label].append(filename)
                    else:
                        # 否则，创建一个新的列表
                        organized_data[label] = [filename]
                else:
                    print(f"跳过无效行: {line.strip()}")

        # 按标签的数值大小排序
        sorted_labels = sorted(organized_data.keys(), key=lambda x: int(x))

        """# 打印结果
        print("整理后的文件:")
        for label in sorted_labels:
            print(f"标签 {label}:")
            for filename in organized_data[label]:
                print(f"  {filename}")
            print()"""

        # 如果需要保存结果到文件，可以取消以下注释
        # save_organized_data(sorted_labels, organized_data)

    except FileNotFoundError:
        print(f"错误: 无法找到文件 '{input_file_path}'")
    except Exception as e:
        print(f"处理文件时出错: {e}")
    return organized_data

# 如果需要保存结果到文件，可以使用以下函数
def save_organized_data(sorted_labels, organized_data, output_file_path='organized_files.txt'):
    with open(output_file_path, 'w') as file:
        for label in sorted_labels:
            file.write(f"标签 {label}:\n")
            for filename in organized_data[label]:
                file.write(f"  {filename}\n")
            file.write("\n")
    print(f"已将结果保存到文件: {output_file_path}")
    
    
input_file = "val.txt"  
sort=organize_files_by_label(input_file)
# 将所有的.png改为.JPEG
for label in sort:
    sort[label] = [filename.replace('.png', '.JPEG') for filename in sort[label]]

In [38]:
import os
import torch
from PIL import Image
from torchvision import transforms, models

# 检查CUDA可用性
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def find_max_activation_image(target_label, layer_idx, node_idx, model, sort, input_base, preprocess):
    # 获取目标标签对应的文件名列表
    filenames = sort.get(str(target_label), [])
    if not filenames:
        raise ValueError(f"No images found for label {target_label}")
    
    # 构建完整路径列表
    image_paths = [os.path.join(input_base, fname) for fname in filenames]
    
    # 收集模型层（确保模型在GPU）
    model = model.to(device)
    layers = []
    def collect_layers(module, prefix=""):
        for name, child in module.named_children():
            if len(list(child.children())) == 0:
                layers.append((f"{prefix}{name}", child))
            else:
                collect_layers(child, prefix=f"{prefix}{name}.")
    collect_layers(model)
    
    # 注册钩子（自动处理GPU数据）
    target_layer = layers[layer_idx][1]
    activation = {}
    def hook(module, input, output):
        activation['output'] = output.detach().to(device)  # 确保输出在GPU
    handle = target_layer.register_forward_hook(hook)
    
    # 预处理参数（创建在GPU）
    white_value = torch.tensor([
        (1.0 - 0.485)/0.229,
        (1.0 - 0.456)/0.224,
        (1.0 - 0.406)/0.225
    ], device=device).view(1, 3, 1, 1)  # 添加device参数
    
    inv_normalize = transforms.Normalize(
        mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
        std=[1/0.229, 1/0.224, 1/0.225]
    )
    
    try:
        # 创建以标签、层和节点命名的输出目录
        output_base = f"output_images_{target_label}_layer{layer_idx}_node{node_idx}"
        os.makedirs(output_base, exist_ok=True)
        
        for img_path in image_paths:
            try:
                # 图像预处理（保持数据在GPU）
                img = Image.open(img_path).convert('RGB')
                input_tensor = preprocess(img).unsqueeze(0).to(device)  # 移动到GPU
                white_tensor = torch.ones_like(input_tensor, device=device) * white_value
                
                max_activation = -float('inf')
                max_image = None
                
                # 滑动窗口生成掩码（使用CUDA加速）
                with torch.cuda.amp.autocast():  # 混合精度加速
                    flag=80
                    for i in range(0, 224 - flag + 1):
                        for j in range(0, 224 - flag + 1):
                            masked = white_tensor.clone()
                            masked[..., i:i+flag, j:j+flag] = input_tensor[..., i:i+flag, j:j+flag]
                            
                            with torch.no_grad():
                                model(masked)
                            
                            output = activation['output']
                            # 保持计算在GPU
                            if output.ndim == 4:
                                act = output[0, node_idx].mean()
                            elif output.ndim == 2:
                                act = output[0, node_idx]
                            else:
                                continue
                            
                            # 比较前移到GPU
                            if act > max_activation:
                                max_activation = act
                                # 延迟数据回传
                                processed_img = inv_normalize(masked.squeeze().cpu())
                                max_image = transforms.ToPILImage()(processed_img)
                                
                                # 释放中间变量内存
                                del masked, output
                                torch.cuda.empty_cache()           
                
                # 保存结果
                if max_image:
                    os.makedirs(output_base, exist_ok=True)
                    output_path = os.path.join(output_base, os.path.basename(img_path))
                    max_image.save(output_path)
                    print(f"Saved result for {img_path} to {output_path}")
                
            except Exception as e:
                print(f"Error processing {img_path}: {e}")
            finally:
                # 显式释放大内存
                del input_tensor, white_tensor
                torch.cuda.empty_cache()
    finally:
        handle.remove()

# 初始化模型（直接在GPU创建）
model = models.resnet18(pretrained=True).to(device)
model.eval()

# 预处理保持数据在GPU
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                       [0.229, 0.224, 0.225])
])


Using device: cuda


In [39]:
for i,label in enumerate(labelselect):
    label=str(label)
    layer=data[i][0]
    node=data[i][1]
    print(label,layer,node)
    
    # 示例调用（添加内存监控）
    with torch.cuda.device(device):
        find_max_activation_image(
            target_label=label,
            layer_idx=layer,
            node_idx=node,
            model=model,
            sort=sort,
            input_base=input_base,
            preprocess=preprocess
        )

928 51 1


KeyboardInterrupt: 